In [1]:
import numpy as np

Linear calibration

In [3]:
M_ = np.loadtxt("data/lin_cal_mat.csv")
print(M_)

[[ 1.25707872e-01 -6.28539361e-02 -6.28539361e-02]
 [ 2.45218238e-18  1.08866211e-01 -1.08866211e-01]
 [ 1.88561808e-01  1.88561808e-01  1.88561808e-01]]


In [ ]:
theta = np.linspace(0, np.pi / 2, 51)
B = []

for th in theta:
    B.append(1 * np.array([np.sin(th), 0, np.cos(th)]))

B = np.array(B)